In [1]:
import sys
from pyspark import SparkContext

sc = SparkContext ( )

In [7]:
from pyspark.sql.types import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

wikiCategoryFile="gs://metcs777/wiki-categorylinks.csv.bz2"
wikiCategoryLinks=sc.textFile(wikiCategoryFile)
wikiCats=wikiCategoryLinks.map(lambda x: x.split(",")).map(lambda x: (x[0].replace('"',''), x[1].replace('"','') ))
df=sqlContext.createDataFrame(wikiCats)

df.show()

+------+--------------------+
|    _1|                  _2|
+------+--------------------+
|434042|   1987_debut_albums|
|434042|Albums_produced_b...|
|434042|Articles_with_hAu...|
|434042|Articles_with_sho...|
|434042|CS1_German-langua...|
|434042|Cacophony_(band)_...|
|434042| Jason_Becker_albums|
|434042|Marty_Friedman_al...|
|434042|Shrapnel_Records_...|
|455070|All_articles_need...|
|455070|All_articles_with...|
|455070|Articles_needing_...|
|455070|Articles_with_OS_...|
|455070|Articles_with_dea...|
|455070|Articles_with_dea...|
|455070|Articles_with_per...|
|455070|Borough_of_Broxbo...|
|455070|            Cheshunt|
|455070|Commons_category_...|
|455070|Coordinates_on_Wi...|
+------+--------------------+
only showing top 20 rows



In [8]:
"""
task3.1
"""
from pyspark.sql import functions as func
from pyspark.sql import DataFrameStatFunctions as statFunc
from pyspark.sql import functions as F
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
df_cate = df.groupBy (df[1]).count()
max = df_cate.agg(func.max("count")).show()
avg = df_cate.agg(func.mean("count")).show()
med = F.expr('percentile_approx(count, 0.5)')
median = df_cate.agg(med.alias('med(count)')).show()
std = df_cate.select(_stddev(col('count')).alias('std')).show()

+----------+
|max(count)|
+----------+
|       273|
+----------+

+------------------+
|        avg(count)|
+------------------+
|1.9264644205228576|
+------------------+

+----------+
|med(count)|
+----------+
|         1|
+----------+

+-----------------+
|              std|
+-----------------+
|8.432165500551477|
+-----------------+



In [9]:
"""
task3.2
"""
top = df_cate.orderBy("count",ascending=[0]).show(10)

+--------------------+-----+
|                  _2|count|
+--------------------+-----+
|Articles_with_sho...|  273|
|All_articles_with...|  222|
|Wikipedia_article...|  212|
|Wikipedia_article...|  212|
|Wikipedia_article...|  196|
|Commons_category_...|  178|
|Wikipedia_article...|  157|
|Wikipedia_article...|  155|
|All_articles_need...|  154|
|Webarchive_templa...|  140|
+--------------------+-----+
only showing top 10 rows



In [27]:
"""
task3.3
"""
top_cate = df_cate.orderBy("count",ascending=[0]).limit(10)
top_page = top_cate.join(df,df[1]==top_cate[0]).drop("count")
top_id = top_page.groupBy(top_page[1]).count()
print(top_id.select(top_id[0]).show())



+------+
|    _1|
+------+
|455070|
|455790|
|456150|
|434000|
|429031|
|429164|
|455361|
|433488|
|433664|
|432147|
|429110|
|432184|
|433593|
|432097|
|455880|
|433663|
|456310|
|433824|
|456234|
|429135|
+------+
only showing top 20 rows

None
